<a href="https://colab.research.google.com/github/damianwgriggs/Quantum-NBA-Sports-Betting/blob/main/NBA_Game_Predictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# ------------------------------------------------------------------------------
# 🚀 FINAL CORRECTED "SUPER CELL" (ROBUST DATA FIX)
# ------------------------------------------------------------------------------
import os
import sys

# --- STEP 1: FORCE-INSTALL PYTHON 3.9 ENVIRONMENT ---
# This prevents the "Version Conflict" errors by creating a clean sandbox.
print("⏳ CHECKING QUANTUM ENVIRONMENT...")
if not os.path.exists('/usr/local/envs/quantum_env'):
    print("   (First run detected: Installing Python 3.9... This takes ~90 seconds)")
    os.system('wget -qO mini.sh https://repo.anaconda.com/miniconda/Miniconda3-py39_23.3.1-0-Linux-x86_64.sh')
    os.system('chmod +x mini.sh')
    os.system('bash ./mini.sh -b -f -p /usr/local > /dev/null')
    os.system('/usr/local/bin/conda create -y -n quantum_env python=3.9 > /dev/null')
    os.system('/usr/local/envs/quantum_env/bin/pip install -q tensorflow==2.15.0 tensorflow-quantum==0.7.3 cirq==1.3.0 sympy==1.12 nba_api pandas requests tqdm > /dev/null')
print("✅ ENVIRONMENT READY.")

# --- STEP 2: WRITE THE ROBUST SCRIPT ---
script_content = """
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_quantum as tfq
import cirq
import sympy
from nba_api.stats.endpoints import leaguegamefinder

# 1. DOWNLOAD DATA (Using the MASTER file now)
print("\\n🏀 DOWNLOADING LATEST NBA DATA...")
# Use the 'Master' file which contains ALL history + current season
elo_url = "https://raw.githubusercontent.com/Neil-Paine-1/NBA-elo/master/nba_elo.csv"
try:
    elo_df = pd.read_csv(elo_url)
    elo_df['date'] = pd.to_datetime(elo_df['date'])
except:
    print("❌ Error: Could not download Elo Data.")
    exit()

# Get Schedule
gamefinder = leaguegamefinder.LeagueGameFinder(league_id_nullable='00')
all_games = gamefinder.get_data_frames()[0]
all_games['GAME_DATE'] = pd.to_datetime(all_games['GAME_DATE'])

# Filter: Last 3 Years to ensure we have enough training data
start_date = pd.to_datetime('2023-10-01')
games = all_games[
    (all_games['GAME_DATE'] >= start_date) &
    (all_games['WL'].notna())
].sort_values('GAME_DATE')

print(f"   Found {len(games)} games in schedule since Oct 2023.")

# 2. PROCESS DATA
def calculate_rest(team_id, game_date, all_games_df):
    team_games = all_games_df[all_games_df['TEAM_ID'] == team_id]
    previous = team_games[team_games['GAME_DATE'] < game_date]
    if previous.empty: return 7
    return min((game_date - previous.iloc[-1]['GAME_DATE']).days, 7)

processed_rows = []
unique_game_ids = games['GAME_ID'].unique()

for gid in unique_game_ids:
    g_data = games[games['GAME_ID'] == gid]
    if len(g_data) < 2: continue

    team_a, team_b = g_data.iloc[0], g_data.iloc[1]

    # Robust Elo Merge
    row_elo = elo_df[(elo_df['date'] == team_a['GAME_DATE']) & (elo_df['team1'] == team_a['TEAM_ABBREVIATION'])]
    if row_elo.empty: continue

    elo_a = row_elo.iloc[0]['elo1_pre']
    elo_b = row_elo.iloc[0]['elo2_pre']

    processed_rows.append({
        'Date': team_a['GAME_DATE'],
        'Team_A': team_a['TEAM_ABBREVIATION'],
        'Team_B': team_b['TEAM_ABBREVIATION'],
        'Q_Elo_A': elo_a / 2000.0,
        'Q_Elo_B': elo_b / 2000.0,
        'Q_Rest_A': calculate_rest(team_a['TEAM_ID'], team_a['GAME_DATE'], games) / 7.0,
        'Q_Rest_B': calculate_rest(team_b['TEAM_ID'], team_b['GAME_DATE'], games) / 7.0,
        'Winner': 1 if team_a['WL'] == 'W' else 0
    })

df = pd.DataFrame(processed_rows)
if df.empty:
    print("❌ Error: Dataframe is empty after processing. Check connection/dates.")
    exit()

print(f"✅ Successfully processed {len(df)} games for Quantum Training.")

# 3. SPLIT DATA (The Robust Fix)
# Instead of 'Today - 30 days', we use 'Last available date - 30 days'
# This prevents empty test sets if the data is slightly old.
last_date_in_data = df['Date'].max()
cutoff_date = last_date_in_data - pd.Timedelta(days=30)

train_df = df[df['Date'] < cutoff_date]
test_df = df[df['Date'] >= cutoff_date]

print(f"📊 Training on {len(train_df)} games.")
print(f"🧪 Backtesting on {len(test_df)} games (From {cutoff_date.date()} to {last_date_in_data.date()})")

# 4. QUANTUM TRAINING
qubits = [cirq.GridQubit(0, i) for i in range(4)]

def process_to_circuits(dataframe):
    circuits = []
    labels = []
    for _, row in dataframe.iterrows():
        c = cirq.Circuit()
        c.append(cirq.rx(row['Q_Elo_A'] * np.pi)(qubits[0]))
        c.append(cirq.rx(row['Q_Elo_B'] * np.pi)(qubits[1]))
        c.append(cirq.ry(row['Q_Rest_A'] * np.pi)(qubits[2]))
        c.append(cirq.ry(row['Q_Rest_B'] * np.pi)(qubits[3]))
        circuits.append(c)
        labels.append(row['Winner'])
    return tfq.convert_to_tensor(circuits), np.array(labels)

X_train, y_train = process_to_circuits(train_df)
X_test, y_test = process_to_circuits(test_df)

# Build & Train
params = sympy.symbols('theta0:4')
q_model_circuit = cirq.Circuit()
q_model_circuit.append(cirq.CNOT(qubits[0], qubits[1]))
q_model_circuit.append(cirq.CNOT(qubits[2], qubits[3]))
for i in range(4): q_model_circuit.append(cirq.rz(params[i])(qubits[i]))

model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(), dtype=tf.string),
    tfq.layers.PQC(q_model_circuit, operators=cirq.Z(qubits[0])),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01), loss='binary_crossentropy', metrics=['accuracy'])
print("\\n🧠 Training Quantum Model...")
model.fit(X_train, y_train, epochs=15, batch_size=32, verbose=0)

# 5. RESULTS
print("\\n💰 BACKTEST RESULTS (Simulated Betting on Test Set)")
print("-" * 65)
print(f"{'Date':<10} | {'Matchup':<15} | {'Pred':<5} | {'Conf%':<5} | {'Result'}")
print("-" * 65)

predictions = model.predict(X_test, verbose=0)
wins, total_bets, balance = 0, 0, 0

for i in range(len(test_df)):
    prob = predictions[i][0]
    if 0.40 <= prob <= 0.60: continue # Skip weak signals

    pred_winner = 1 if prob > 0.5 else 0
    is_correct = (pred_winner == y_test[i])
    total_bets += 1

    if is_correct:
        wins += 1
        balance += 91
        res_str = "WIN 🟢"
    else:
        balance -= 100
        res_str = "LOSS 🔴"

    if i > len(test_df) - 11:
        row = test_df.iloc[i]
        matchup = f"{row['Team_A']} vs {row['Team_B']}"
        conf = prob if prob > 0.5 else (1 - prob)
        print(f"{row['Date'].strftime('%m-%d')}   | {matchup:<15} | {pred_winner}     | {conf:.2f}  | {res_str}")

print("-" * 65)
if total_bets > 0:
    print(f"🎯 Accuracy: {(wins/total_bets*100):.1f}%")
    print(f"💵 Net Profit: ${balance}")
else:
    print("⚠️ No bets placed (High Uncertainty).")
"""

with open("run_quantum_bets_v2.py", "w") as f:
    f.write(script_content)

# --- STEP 3: EXECUTE ---
print("🚀 LAUNCHING SCRIPT...")
!/usr/local/envs/quantum_env/bin/python run_quantum_bets_v2.py

⏳ CHECKING QUANTUM ENVIRONMENT...
✅ ENVIRONMENT READY.
🚀 LAUNCHING SCRIPT...
2025-11-18 18:29:43.370909: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-11-18 18:29:43.370974: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-11-18 18:29:43.372409: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-11-18 18:29:44.436756: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT

🏀 DOWNLOADING LATEST NBA DATA...
   Found 6152 games in schedule since Oct 2023.
✅ Successfully processed 2120 games for Quantum Training.
📊 Training on 1930 games

In [3]:
!/usr/local/envs/quantum_env/bin/python run_quantum_bets.py

2025-11-18 18:25:49.014893: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-11-18 18:25:49.014981: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-11-18 18:25:49.016821: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-11-18 18:25:50.105073: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT

🏀 PART 1: DOWNLOADING & PROCESSING NBA DATA...
✅ Data Processed: 2311 games found.

🧠 PART 2: TRAINING QUANTUM NEURAL NETWORK...
✅ Training Complete.

💰 PART 3: BACKTEST RESULTS (Last 30 Days)
-----------------------------------------------